In [45]:
from keras_contrib.layers import CRF
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras.backend import categorical_crossentropy
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

### Necessary Functions

In [54]:
def get_sentences(dataset):
    n_sent = 1
    # print(dataset[0])
    grouped = dataset.groupby("sentence_idx").apply(lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                                                      s["tag"].values.tolist())])
    sentences = [s for s in grouped]

    def get_next():
        nonlocal n_sent
        try:
            s = grouped["Sentence: {}".format(n_sent)]
            n_sent += 1
            return s
        except:
            return None

    return sentences, get_next


def get_max_len(sentences):
    return max([len(s) for s in sentences])


def add_sentence_id_column(dataset):
    sentence_idx = 1
    sentence_indices = []
    for word in dataset['word']:
        sentence_indices.append(sentence_idx)
        if word == '.':
            sentence_idx += 1
    dataset.insert(0, 'sentence_idx', sentence_indices)
    return dataset

In [70]:
tag2idx = {'O':0,'I-per':1, 'I-eve':2, 'B-geo':3, 'I-geo':4, 'B-tim':5, 'B-org': 6, 'B-eve':7, 'B-nat':8, 'I-nat':9, 'B-per':10, 'B-art':11, 'B-gpe':12, 'I-art':13, 'I-tim':14, 'I-org':15, 'I-gpe':16}
n_tags = len(tag2idx.values())
def encoding(data):
    # dframe = lstmFun.add_sentence_id_column(self.data)
    sentences, get_next = get_sentences(data)
    maxlen = get_max_len(sentences)
    print('Maximum sequence length:', maxlen)

    words = list(set(data["word"].values))
    words = ["ENDPAD"] + words
    n_words = len(words);
    n_tags = len(tag2idx.values())

    word2idx = {w: i for i, w in enumerate(words)}

    x = [[word2idx[w[0]] for w in s] for s in sentences]
    x = pad_sequences(maxlen=maxlen, sequences=x, padding="post", value=n_words - 1)

    y = [[tag2idx[w[1]] for w in s] for s in sentences]
    y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tag2idx["O"])
    y = [to_categorical(i, num_classes=n_tags) for i in y]
    return x, y, maxlen, n_words

### encoding train and test data

In [56]:
train_data = pd.read_csv("work/data/label_data/ner_train_data.csv", encoding = "ISO-8859-1")
test_data = pd.read_csv("work/data/label_data/ner_test_data.csv", encoding = "ISO-8859-1")
x_train, y_train, train_maxlen, train_n_words = encoding(train_data)
x_test, y_test, test_maxlen, test_n_words= encoding(test_data)

Maximum sequence length: 104
Maximum sequence length: 104


### Train the model

In [88]:
input_t = Input(shape=(train_maxlen,))
model = Embedding(input_dim=train_n_words, output_dim=100, input_length=train_maxlen)(input_t)
model = Dropout(0.1)(model)
model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
out = TimeDistributed(Dense(n_tags, activation="softmax"))(model)  # softmax output layer
# model = Bidirectional(LSTM(units=100, return_sequences=True, recurrent_dropout=0.1))(model)
# crf = CRF(n_tags)  # CRF layer with n_tags output units
# out = crf(model)  # Apply CRF layer to the output of LSTM
model = Model(input_t, out)

In [50]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.layers import Input
from keras.models import Model

In [89]:
from keras.losses import CategoricalCrossentropy
def custom_loss(y_true, y_pred):
    penalty_weight = tf.constant(10.0)

    # Calculate the loss
    loss = - tf.reduce_sum(y_true * tf.math.log(y_pred), axis=-1)
    print("loss", loss.shape)

    mask = tf.equal(tf.equal(y_true[:, :, 0] , 1), tf.less(y_pred[:, :, 0] , 0.7))
    mask = tf.cast(mask, tf.float32)
    print("mask", mask)

    loss = loss * tf.multiply(penalty_weight, mask)

    return loss
import tensorflow as tf
from tensorflow.keras.losses import CategoricalCrossentropy

cross_entropy_loss = CategoricalCrossentropy()
model = Model(input_t, out)
model.compile(optimizer="adam", loss=cross_entropy_loss, metrics=["categorical_accuracy"])

In [79]:
 # custom_loss(y_test, preds)

In [90]:
history = model.fit(x_train, np.array(y_train), batch_size=32, epochs=1, verbose=1)

1199/1199 [==============================] - 170s 139ms/step - loss: 0.0913 - categorical_accuracy: 0.9810


In [93]:
# Save Model
dir = "/home/jovyan/work/data/output/LSTM_Final"
model.save(dir)

2023-06-13 23:57:58.803438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'while/Placeholder_2' with dtype float and shape [?,100]
	 [[{{node while/Placeholder_2}}]]
2023-06-13 23:57:58.996781: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'while/Placeholder_2' with dtype float and shape [?,100]
	 [[{{node while/Placeholder_2}}]]
2023-06-13 23:57:59.043772: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and 

INFO:tensorflow:Assets written to: /home/jovyan/work/data/output/LSTM_Final/assets


INFO:tensorflow:Assets written to: /home/jovyan/work/data/output/LSTM_Final/assets


In [ ]:
# Evaluation
loss, accuracy = model.evaluate(x_test, np.array(y_test))
print("Loss: {:.4f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy * 100))

### Evaluation per tag

In [48]:
# Read Model
lstm_1_epochs = 'work/data/output/LSTM_1batch'
model = tf.keras.models.load_model(lstm_1_epochs)

In [91]:
# Get the predicted labels
y_pred = model.predict(x_test)
y_pred_indices = np.argmax(y_pred, axis=-1)

y_pred_flattened = y_pred_indices.flatten()
# Y_train_flatten = y_train_indices.flatten()

1199/1199 [==============================] - 32s 27ms/step


In [92]:
unique_values, counts = np.unique(y_pred_flattened, return_counts=True)

for value, count in zip(unique_values, counts):
    print("Value:", value, "Count:", count)

[0 0 0 ... 0 0 0]
Value: 0 Count: 3866582
Value: 1 Count: 13758
Value: 3 Count: 34278
Value: 4 Count: 5126
Value: 5 Count: 15390
Value: 6 Count: 12461
Value: 10 Count: 12798
Value: 12 Count: 12519
Value: 14 Count: 4073
Value: 15 Count: 13183


In [94]:
y_true_indices = np.argmax(y_test, axis=-1)
y_true_flattened = y_true_indices.flatten()

In [67]:
print(y_true_flattened)
unique_values, counts = np.unique(y_true_flattened, return_counts=True)

for value, count in zip(unique_values, counts):
    print("Value:", value, "Count:", count)

[0 0 0 ... 0 0 0]
Value: 0 Count: 3861559
Value: 1 Count: 13806
Value: 2 Count: 200
Value: 3 Count: 29980
Value: 4 Count: 5964
Value: 5 Count: 16284
Value: 6 Count: 16230
Value: 7 Count: 248
Value: 8 Count: 151
Value: 9 Count: 39
Value: 10 Count: 13601
Value: 11 Count: 316
Value: 12 Count: 12695
Value: 13 Count: 239
Value: 14 Count: 5228
Value: 15 Count: 13469
Value: 16 Count: 159


In [159]:
precision_per_tag = {}
recall_per_tag = {}
f1_per_tag = {}
for tag, id in tag2idx.items():
    true_positives = np.sum((y_pred_flattened == id) & (y_true_flattened == id))
    false_positives = np.sum((y_pred_flattened == id) & (y_true_flattened != id))
    false_negatives = np.sum((y_pred_flattened != id) & (y_true_flattened == id))

    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    precision_per_tag[tag] = round(precision, 2) * 100
    recall_per_tag[tag] = round(recall, 2) * 100
    f1_per_tag[tag] = round(f1_score, 2) * 100


for tag, id in tag2idx.items():
    print("Tag:", tag)
    print("Precision: ", precision_per_tag[tag])
    print("F1 Score:", f1_per_tag[tag])
    print("recall:", recall_per_tag[tag])
    print()



Tag: O
Precision:  100.0
F1 Score: 100.0
recall: 100.0

Tag: I-per
Precision:  87.0
F1 Score: 87.0
recall: 86.0

Tag: I-eve
Precision:  0
F1 Score: 0
recall: 0.0

Tag: B-geo
Precision:  80.0
F1 Score: 86.0
recall: 92.0

Tag: I-geo
Precision:  81.0
F1 Score: 75.0
recall: 70.0

Tag: B-tim
Precision:  90.0
F1 Score: 88.0
recall: 85.0

Tag: B-org
Precision:  81.0
F1 Score: 70.0
recall: 62.0

Tag: B-eve
Precision:  0
F1 Score: 0
recall: 0.0

Tag: B-nat
Precision:  0
F1 Score: 0
recall: 0.0

Tag: I-nat
Precision:  0
F1 Score: 0
recall: 0.0

Tag: B-per
Precision:  86.0
F1 Score: 83.0
recall: 81.0

Tag: B-art
Precision:  0
F1 Score: 0
recall: 0.0

Tag: B-gpe
Precision:  94.0
F1 Score: 93.0
recall: 93.0

Tag: I-art
Precision:  0
F1 Score: 0
recall: 0.0

Tag: I-tim
Precision:  83.0
F1 Score: 72.0
recall: 64.0

Tag: I-org
Precision:  77.0
F1 Score: 76.0
recall: 75.0

Tag: I-gpe
Precision:  0
F1 Score: 0
recall: 0.0



In [160]:
# Calculate overall precision, recall, and F1 score
true_positives = np.sum((y_pred_flattened == y_true_flattened) & (y_true_flattened != 0))
false_positives = np.sum((y_pred_flattened != y_true_flattened) & (y_true_flattened != 0))
false_negatives = np.sum((y_pred_flattened != y_true_flattened) & (y_true_flattened == 0))

precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print("Precision: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F1 Score: {:.2f}".format(f1_score))

Precision: 0.81
Recall: 0.97
F1 Score: 0.88


#### Read models and  Compair two models with different epoch

In [19]:
import tensorflow as tf
lstm_1_epochs = 'work/data/output/LSTM_1batch'
model1e = tf.keras.models.load_model(lstm_1_epochs)


In [20]:
loss, accuracy = model1e.evaluate(x_test, np.array(y_test))
print("Loss: {:.4f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy * 100))

1199/1199 [==============================] - 46s 38ms/step - loss: 13.4553 - accuracy: 0.0038
Loss: 13.4553
Accuracy: 0.38%


In [29]:
lstm_40_epochs = 'work/data/output/LSTM_'
model40e = tf.keras.models.load_model(lstm_40_epochs)
loss, accuracy = model1e.evaluate(x_test, np.array(y_test))
print("Loss: {:.4f}".format(loss))
print("Accuracy: {:.2f}%".format(accuracy * 100))


1199/1199 [==============================] - 33s 27ms/step - loss: 0.0254 - accuracy: 0.9927
Loss: 0.0254
Accuracy: 99.27%
